# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

pd.options.mode.chained_assignment = None  # default='warn'

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
#load the data from csv
csv_file = "output_data/cities.csv"

weather_df = pd.read_csv(csv_file)
weather_df


,Countries,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
0,MG,Ambilobe,-13.2000,49.0500,1614222732,295.30,91,49,1.43
1,US,Barrow,71.2906,-156.7887,1614222734,240.15,68,1,6.17
2,GL,Upernavik,72.7868,-56.1549,1614222737,259.38,54,63,1.80
3,AR,Ushuaia,-54.8000,-68.3000,1614222740,279.15,70,40,11.32
4,CL,Punta Arenas,-53.1500,-70.9167,1614222741,282.25,77,88,4.31
...,...,...,...,...,...,...,...,...,...
556,TJ,Vorukh,39.8494,70.5838,1614223542,267.23,77,100,2.72
557,US,Beckley,37.7782,-81.1882,1614223544,285.74,47,90,5.14
558,PT,Vila,42.0304,-8.1588,1614223546,281.48,94,100,0.89
559,PE,Lima,-12.0432,-77.0282,1614223547,294.80,83,0,4.12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
# gmaps config
gmaps.configure(api_key = g_key)

# Use the lat and Lon columns as the locations variable
locations = weather_df[["Lat", "Lon"]]

# Use the humidity % column of the df as the weight variable
humidity = weather_df["Humidities %"]
humidity

0      91
1      68
2      54
3      70
4      77
       ..
556    77
557    47
558    94
559    83
560    79
Name: Humidities %, Length: 561, dtype: int64

In [32]:
#heatmap
fig = gmaps.figure(center=(25, 15), zoom_level = 1.8)

# Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 1.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
#Drop null values
ideal = weather_df.dropna(how='all')
ideal

,Countries,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
0,MG,Ambilobe,-13.2000,49.0500,1614222732,295.30,91,49,1.43
1,US,Barrow,71.2906,-156.7887,1614222734,240.15,68,1,6.17
2,GL,Upernavik,72.7868,-56.1549,1614222737,259.38,54,63,1.80
3,AR,Ushuaia,-54.8000,-68.3000,1614222740,279.15,70,40,11.32
4,CL,Punta Arenas,-53.1500,-70.9167,1614222741,282.25,77,88,4.31
...,...,...,...,...,...,...,...,...,...
556,TJ,Vorukh,39.8494,70.5838,1614223542,267.23,77,100,2.72
557,US,Beckley,37.7782,-81.1882,1614223544,285.74,47,90,5.14
558,PT,Vila,42.0304,-8.1588,1614223546,281.48,94,100,0.89
559,PE,Lima,-12.0432,-77.0282,1614223547,294.80,83,0,4.12


In [34]:
#I notice the temp needs to some converstions but i wont change it now. I will just use the data the way it is for now. 
ideal_df = ideal.loc[(ideal["Max Temp (F)"] < 400) & (ideal["Max Temp (F)"] > 300)]
ideal_df

# narrow the list by selecting ideal wind speeds 
wind_speed = ideal_df.loc[(ideal_df["Wind Speed"] < 5)]
wind_speed

#narrow the humiditie levels
new_ideal_df = wind_speed.loc[(wind_speed["Humidities %"] < 70) & (wind_speed["Max Temp (F)"] > 50)]
new_ideal_df

,Countries,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
32,AU,Mount Isa,-20.7333,139.5000,1614222781,307.15,14,0,3.09
157,PH,Catuday,16.2923,119.8062,1614222960,300.61,68,83,2.85
166,PF,Tautira,-17.7333,-149.1500,1614222973,303.15,66,75,3.60
195,ID,Pangkalanbuun,-2.6833,111.6167,1614223013,304.65,50,44,2.54
234,MY,George Town,5.4112,100.3354,1614222947,301.47,69,20,3.09
257,PH,Dingle,10.9995,122.6711,1614223100,302.30,66,32,4.64
262,ID,Nabire,-3.3667,135.4833,1614223108,303.75,62,31,2.50
274,IN,Kavaratti,10.5669,72.6420,1614223125,300.73,67,88,4.10
283,MY,Kota Bharu,6.1333,102.2386,1614223138,301.53,65,20,2.57
301,AU,Karratha,-20.7377,116.8463,1614223162,309.34,36,0,0.94


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = new_ideal_df
hotel_df

,Countries,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
32,AU,Mount Isa,-20.7333,139.5000,1614222781,307.15,14,0,3.09
157,PH,Catuday,16.2923,119.8062,1614222960,300.61,68,83,2.85
166,PF,Tautira,-17.7333,-149.1500,1614222973,303.15,66,75,3.60
195,ID,Pangkalanbuun,-2.6833,111.6167,1614223013,304.65,50,44,2.54
234,MY,George Town,5.4112,100.3354,1614222947,301.47,69,20,3.09
257,PH,Dingle,10.9995,122.6711,1614223100,302.30,66,32,4.64
262,ID,Nabire,-3.3667,135.4833,1614223108,303.75,62,31,2.50
274,IN,Kavaratti,10.5669,72.6420,1614223125,300.73,67,88,4.10
283,MY,Kota Bharu,6.1333,102.2386,1614223138,301.53,65,20,2.57
301,AU,Karratha,-20.7377,116.8463,1614223162,309.34,36,0,0.94


In [36]:
#create a new colum of hotel names
hotel_df["Hotel Names"] = ""
hotel_df

,Countries,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed,Hotel Names
32,AU,Mount Isa,-20.7333,139.5000,1614222781,307.15,14,0,3.09,
157,PH,Catuday,16.2923,119.8062,1614222960,300.61,68,83,2.85,
166,PF,Tautira,-17.7333,-149.1500,1614222973,303.15,66,75,3.60,
195,ID,Pangkalanbuun,-2.6833,111.6167,1614223013,304.65,50,44,2.54,
234,MY,George Town,5.4112,100.3354,1614222947,301.47,69,20,3.09,
257,PH,Dingle,10.9995,122.6711,1614223100,302.30,66,32,4.64,
262,ID,Nabire,-3.3667,135.4833,1614223108,303.75,62,31,2.50,
274,IN,Kavaratti,10.5669,72.6420,1614223125,300.73,67,88,4.10,
283,MY,Kota Bharu,6.1333,102.2386,1614223138,301.53,65,20,2.57,
301,AU,Karratha,-20.7377,116.8463,1614223162,309.34,36,0,0.94,


In [37]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [39]:
# api request the hotels within the param 

for index, row in hotel_df.iterrows():
    
    # lat and lon from df
    lat = row["Lat"]
    lon = row["Lon"]

    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    print(f"Retrieving Results by {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    #results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results by 32: Mount Isa.
Closest hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results by 157: Catuday.
Closest hotel is Hotel G Beach Resort and Restobar.
------------
Retrieving Results by 166: Tautira.
Missing field/result... skipping.
------------
Retrieving Results by 195: Pangkalanbuun.
Closest hotel is Arsela Hotel Pangkalan Bun.
------------
Retrieving Results by 234: George Town.
Closest hotel is Eastern & Oriental Hotel, Penang.
------------
Retrieving Results by 257: Dingle.
Missing field/result... skipping.
------------
Retrieving Results by 262: Nabire.
Closest hotel is Hotel Grand Papua Nabire.
------------
Retrieving Results by 274: Kavaratti.
Closest hotel is yaad Hotal.
------------
Retrieving Results by 283: Kota Bharu.
Closest hotel is Perdana Kota Bharu (Formerly known as Hotel Perdana Kota Bharu).
------------
Retrieving Results by 301: Karratha.
Closest hotel is Best Western Plus The Ranges Karratha.
------------
Retrieving Results by 306

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add a layer to the map
fig.add_layer(markers)

# Display figure
fig